# 1.0 - IMPORTS

In [66]:
# Adicione o caminho do diretório raiz do projeto
import os, sys; sys.path.append(os.path.abspath('../'))

import pandas as pd
import numpy as np
import importlib
import plotly.express as px

from packages import etl
from packages import utils
from packages import layouts

importlib.reload(etl.extract)
importlib.reload(etl.load)
importlib.reload(etl.transform)
importlib.reload(utils)
importlib.reload(layouts.cities)
importlib.reload(layouts.countries)
importlib.reload(layouts.cuisines)

pd.set_option('display.max_columns', None);

# 2.0 - ETL

In [12]:
# Extrair os dados
df_raw = etl.extract.csv('../data/raw/zomato.csv')

# Transformar os dados
df = etl.transform.dataframe(df_raw)

# Salvar os dados transformados (opcional)
etl.load.save(df, '../data/processed/zomato.csv')

# Visualizar os dados
df.head()

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,price_range,aggregate_rating,rating_color,rating_text,votes,rating_color_name,rating_name,country_name,price_type
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,3,4.6,3F7E00,Excellent,619,darkgreen,excellent,Philippines,expensive
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,3100,Botswana Pula(P),0,0,0,4,4.7,3F7E00,Excellent,469,darkgreen,excellent,Philippines,gourmet
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,800,Botswana Pula(P),0,0,0,3,4.4,5BA829,Very Good,867,green,great,Philippines,expensive
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,700,Botswana Pula(P),0,0,0,3,4.4,5BA829,Very Good,858,green,great,Philippines,expensive
5,6304833,Manam,162,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.020380,14.552351,Filipino,700,Botswana Pula(P),0,0,0,3,4.7,3F7E00,Excellent,930,darkgreen,excellent,Philippines,expensive


# 3.0 - Desafio

## 3.1 - Geral

### 1. Quantos restaurantes únicos estão registrados?

In [13]:
df.restaurant_id.nunique()

6929

### 2. Quantos países únicos estão registrados?

In [14]:
df.country_name.nunique()

15

### 3. Quantas cidades únicas estão registradas?

In [15]:
df.city.nunique()

125

### 4. Qual o total de avaliações feitas?

In [16]:
df.votes.sum()

4194533

### 5. Qual o total de tipos de culinária registrados?

In [17]:
df.cuisines.nunique()

165

## 3.2 - País

### 1. Qual o nome do país que possui mais cidades registradas?

In [20]:
(df[['country_name', 
    'city']].groupby([
    'country_name'])
    .count()
    .reset_index()
    .sort_values('city', ascending=False)
    .head(1))

,country_name,city
4,India,3111


### 2. Qual o nome do país que possui mais restaurantes registrados?

In [21]:
(df[['country_name', 
    'restaurant_id']].groupby([
    'country_name'])
    .count()
    .reset_index()
    .sort_values('restaurant_id', ascending=False)
    .head(1))

,country_name,restaurant_id
4,India,3111


### 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?

In [22]:
df_aux = df[df['price_range'] == 4]

(df_aux[['country_name', 'city']]
 .groupby(['country_name'])
 .count()
 .reset_index()
 .sort_values(['city'], ascending=False)
 .head(1))

,country_name,city
14,United States of America,415


### 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [23]:
(df_aux[['country_name', 'cuisines']]
 .groupby(['country_name'])
 .count()
 .reset_index()
 .sort_values(['cuisines'], ascending=False)
 .head(1))

,country_name,cuisines
14,United States of America,415


### 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [24]:
(df_aux[['country_name', 'votes']]
 .groupby(['country_name'])
 .sum()
 .reset_index()
 .sort_values(['votes'], ascending=False)
 .head(1))

,country_name,votes
4,India,169150


### 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [25]:
df__delivery = df[df['is_delivering_now'] == 1]
(df__delivery[['country_name', 'is_delivering_now']]
 .groupby(['country_name'])
 .count()
 .reset_index()
 .sort_values(['is_delivering_now'], ascending=False)
 .head(1))

,country_name,is_delivering_now
0,India,1150


### 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [27]:
df_aux = df[df['has_table_booking'] == 1]

(df_aux[['country_name', 'has_table_booking']]
 .groupby(['country_name'])
 .count()
 .reset_index()
 .sort_values(['has_table_booking'], ascending=False)
 .head(1))

,country_name,has_table_booking
2,India,256


### 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [30]:
df_avg_votes = (df_aux[['country_name', 'votes']]
          .groupby(['country_name'])
          .mean()
          .reset_index()
          .sort_values(['votes'], ascending=False))
df_avg_votes.columns = ['country_name', 'avg_votes']
df_avg_votes.head(1)

,country_name,avg_votes
2,India,2110.742188


### 9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [31]:
(df[['country_name', 'aggregate_rating']]
 .groupby(['country_name'])
 .mean()
 .sort_values('aggregate_rating', ascending=False)
 .reset_index()
 .head(1))

,country_name,aggregate_rating
0,Indonesia,4.60125


### 10. Qual a média de preço de um prato para dois por país?

In [32]:
df.average_cost_for_two.mean()

7520.077211718863

## 3.3 - Cidade

### 1. Qual o nome da cidade que possui mais restaurantes registrados?

In [34]:
(df[['restaurant_id', 'city']]
 .groupby(['city'])
 .count()
 .reset_index()
 .sort_values(['restaurant_id'], ascending=False)
 .head(1))

,city,restaurant_id
0,Abu Dhabi,80


### 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [36]:
df_votes = df[df['votes'] == 4]
(df_votes[['votes', 'city']]
 .groupby(['city'])
 .count()
 .reset_index()
 .sort_values(['votes'], ascending=False)
 .head(1))

,city,votes
6,Ooty,6


### 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [37]:
df_votes = df[df['aggregate_rating'] < 2.5]
(df_votes[['aggregate_rating', 'city']]
 .groupby(['city'])
 .count()
 .reset_index()
 .sort_values(['aggregate_rating'], ascending=False)
 .head(1))

,city,aggregate_rating
6,Gangtok,33


### 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [38]:
(df[['restaurant_id', 'city', 'average_cost_for_two']]
 .groupby(['city', 'average_cost_for_two'])
 .count()
 .sort_values(['average_cost_for_two'], ascending=False)
 .head(1))

,,restaurant_id
city,average_cost_for_two,
Adelaide,25000017,1


### 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [39]:
(df[['city', 'cuisines']]
 .groupby(['city'])
 .count()
 .reset_index()
 .sort_values(['cuisines'], ascending=False)
 .head(1))

,city,cuisines
0,Abu Dhabi,80


### 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [40]:
df_booking = df[df['has_table_booking'] == 1]
(df_booking[['city', 'has_table_booking']]
 .groupby(['city'])
 .count()
 .reset_index()
 .sort_values(['has_table_booking'], ascending=False)
 .head(1))

,city,has_table_booking
8,Bangalore,42


### 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [41]:
df_booking = df[df['is_delivering_now'] == 1]
(df_booking[['city', 'is_delivering_now']]
 .groupby(['city'])
 .count()
 .reset_index()
 .sort_values(['is_delivering_now'], ascending=False)
 .head(1))

,city,is_delivering_now
48,Vadodara,48


### 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [42]:
df_booking = df[df['has_online_delivery'] == 1]
(df_booking[['city', 'has_online_delivery']]
 .groupby(['city'])
 .count()
 .reset_index()
 .sort_values(['has_online_delivery'], ascending=False)
 .head(1))

,city,has_online_delivery
7,Bhopal,75


## 3.4 - Restaurante

### 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [43]:
(df[['restaurant_name', 'votes']]
 .groupby(['restaurant_name'])
 .sum()
 .reset_index()
 .sort_values('votes', ascending=False)
 .head(1))

,restaurant_name,votes
1524,Domino's Pizza,59749


### 2. Qual o nome do restaurante com a maior nota média?

In [44]:
(df[['restaurant_name', 'aggregate_rating']]
 .sort_values('aggregate_rating', ascending=False)
 .head(1))

,restaurant_name,aggregate_rating
811,The Purple Pig,4.9


### 3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [45]:
(df[['restaurant_name', 'average_cost_for_two']]
 .sort_values(['average_cost_for_two'], ascending=False)
 .head(1))

,restaurant_name,average_cost_for_two
385,d'Arry's Verandah Restaurant,25000017


### 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [46]:
df_brazilian = df.loc[df['cuisines'] == 'Brazilian', :].copy()
df_brazilian['index'] = df_brazilian.index
df_brazilian = df_brazilian.reset_index()
(df_brazilian[['index', 'restaurant_name', 'aggregate_rating']]
 .sort_values(['aggregate_rating', 'index'], ascending=[False, True])
 .head(1))

,index,restaurant_name,aggregate_rating
19,207,Braseiro da Gávea,4.9


### 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [47]:
df_brazil = (df.loc[
    (df['cuisines'] == 'Brazilian') & 
    (df['country_name'] == 'Brazil'), :]
    .copy())

df_brazil['index'] = df_brazil.index
df_brazil = df_brazil.reset_index()
(df_brazil[['index', 'country_name', 'restaurant_name', 'aggregate_rating']]
 .sort_values(['aggregate_rating', 'index'], ascending=[False, True])
 .head(1))

,index,country_name,restaurant_name,aggregate_rating
19,207,Brazil,Braseiro da Gávea,4.9


### 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [48]:
df_delivery = df.loc[df['has_online_delivery'] == 1]
(df_delivery[['restaurant_name', 'has_online_delivery','votes']]
 .groupby(['restaurant_name', 'has_online_delivery'], observed=False)
 .mean()
 .sort_values(['votes'], ascending=False)
 .reset_index()
 .head(1))


,restaurant_name,has_online_delivery,votes
0,Bawarchi,1,13903.666667


### 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [51]:
(df[['restaurant_name', 'has_table_booking', 'average_cost_for_two']]
 .groupby(['has_table_booking', 'restaurant_name'], observed=False)
 .mean()
 .sort_values(['average_cost_for_two'], ascending=False)
 .reset_index()
 .head(2))


,has_table_booking,restaurant_name,average_cost_for_two
0,1,d'Arry's Verandah Restaurant,25000017.0
1,0,The Café - Hotel Mulia,1200000.0


### 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [52]:
df_usa_aux = (df[
    (df['country_name'] == 'United States of America') &
    ((df['cuisines'] == 'Japanese') |
    (df['restaurant_name'] == 'Blue Tractor BBQ & Brewery'))
])
(df_usa_aux[['country_name', 
         'cuisines',
         'restaurant_name',
         'average_cost_for_two'
         ]]
        .groupby(['country_name', 
                   'cuisines', 
                   'restaurant_name'
                   ])
        .mean()
        .sort_values([
            'cuisines', 
            'average_cost_for_two'
        ], ascending=[True, False])
        .head(2))

average_cost_for_two
country_name             cuisines restaurant_name                                 
United States of America American Blue Tractor BBQ & Brewery                  25.0
                         Japanese Nobu                                       250.0

## 3.5 - Tipos de Culinária

### 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [81]:
df_italian = df[df['cuisines'] == 'Italian']
layouts.cuisines.top5_restaurants_cuisines(df_italian, ascending=False)

,country_name,cuisines,aggregate_rating,restaurant_name
366,Australia,Italian,4.9,Andre's Cucina & Polenta Bar


### 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [82]:
df_italian = df[df['cuisines'] == 'Italian']
layouts.cuisines.top5_restaurants_cuisines(df_italian, ascending=True)

,country_name,cuisines,aggregate_rating,restaurant_name
6723,England,Italian,0.0,Le Delicatezze Di Bruno


### 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [83]:
df_american = df[df['cuisines'] == 'American']
layouts.cuisines.top5_restaurants_cuisines(df_american, ascending=False)

,country_name,cuisines,aggregate_rating,restaurant_name
726,Canada,American,4.9,OEB Breakfast Co.


### 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [84]:
layouts.cuisines.top5_restaurants_cuisines(df_american, ascending=True)

,country_name,cuisines,aggregate_rating,restaurant_name
6751,England,American,0.0,Alston Bar & Beef


### 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [85]:
df_arabian = df[df['cuisines'] == 'Arabian']
layouts.cuisines.top5_restaurants_cuisines(df_arabian, ascending=False)

,country_name,cuisines,aggregate_rating,restaurant_name
3896,India,Arabian,4.7,Mandi@36


### 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [86]:
layouts.cuisines.top5_restaurants_cuisines(df_arabian, ascending=True)

,country_name,cuisines,aggregate_rating,restaurant_name
337,Brazil,Arabian,0.0,Raful


### 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [87]:
df_japanese = df[df['cuisines'] == 'Japanese']
layouts.cuisines.top5_restaurants_cuisines(df_japanese, ascending=False)

,country_name,cuisines,aggregate_rating,restaurant_name
240,Brazil,Japanese,4.9,Sushi Leblon


### 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [88]:
layouts.cuisines.top5_restaurants_cuisines(df_japanese, ascending=True)

,country_name,cuisines,aggregate_rating,restaurant_name
123,Brazil,Japanese,0.0,Banzai Sushi


### 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [90]:
df_home_made = df[df['cuisines'] == 'Home-made']
layouts.cuisines.top5_restaurants_cuisines(df_home_made, ascending=False)

,country_name,cuisines,aggregate_rating,restaurant_name
7525,Turkey,Home-made,4.0,Kanaat Lokantası


### 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?

In [91]:
layouts.cuisines.top5_restaurants_cuisines(df_home_made, ascending=True)

,country_name,cuisines,aggregate_rating,restaurant_name
7373,Turkey,Home-made,3.7,GurMekan Restaurant


### 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [92]:
df_cuis_filtered = df[['cuisines', 'average_cost_for_two']].copy()
df_cuis_filtered['index'] = df_cuis_filtered.index
df_cuis = (df_cuis_filtered.sort_values(['average_cost_for_two', 'index'], ascending=[False, True])
        .head(1))
df_cuis = df_cuis.drop('index', axis=1)
df_cuis

,cuisines,average_cost_for_two
385,Modern Australian,25000017


### 12. Qual o tipo de culinária que possui a maior nota média?

In [93]:
df_cuis_filtered = df[['cuisines', 'aggregate_rating']].copy()
df_cuis_filtered['index'] = df_cuis_filtered.index
df_cuis = (df_cuis_filtered.sort_values(['aggregate_rating', 'index'], ascending=[False, True])
        .head(1))
df_cuis = df_cuis.drop('index', axis=1)
df_cuis

,cuisines,aggregate_rating
17,Japanese,4.9


### 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [94]:

df_deliv = df[(df['has_online_delivery'] == 1) & (df['is_delivering_now'] == 1)]

df_deliv_result = (df_deliv[['restaurant_id', 'cuisines']]
 .groupby('cuisines')
 .count()
 .sort_values('restaurant_id', ascending=False)
 .reset_index()
 .head(1))
df_deliv_result.columns = ['cuisines', 'total de restaurantes']
df_deliv_result

,cuisines,total de restaurantes
0,North Indian,317
